In [1]:
from pathlib import Path
import os

yr = 2023
d = 3

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def is_number(c):
  numbers = ['0','1','2','3','4','5','6','7','8','9']
  return c in numbers

def is_symbol(c):
  return (c is not None and isinstance(c,str)
  and len(c)==1
  and not is_number(c)
  and c != '.')


def format_input(inp):
  '''Turn text input into a dictionary'''

  d = {}
  for i, l in enumerate(inp.split('\n')):
    for j, c in enumerate(l):
      if c != '.':
          d[(i,j)] = c
  return d


In [3]:
def get_part_numbers(formatted_inp):

  max_i =  max([x[0] for x in formatted_inp.keys()])
  max_j = max([x[1] for x in formatted_inp.keys()])

  def check_adjacent_symbols(i, j):
    for ii in [i-1,i,i+1]:
      for jj in [j-1, j, j+1]:
        if not (ii == i and jj == j):
          if is_symbol(formatted_inp.get((ii,jj), None)):
            return True
    return False

  found_part_numbers = []
  found_part_number_indices = []
  for i in range(max_i+1):
    for j in range(max_j+1):
      if is_number(formatted_inp.get((i,j), None)):
        k = j
        cur_number_chars = []
        cur_number_char_indices = []
        adjacent_symbol = False
        while (is_number(formatted_inp.get((i,k), None))
        and (i,k) not in found_part_number_indices):
          cur_number_chars.append(formatted_inp[(i,k)])
          cur_number_char_indices.append((i,k))
          if(check_adjacent_symbols(i, k)):
            adjacent_symbol = True
          k += 1
        if adjacent_symbol:
          found_part_numbers.append(int(''.join(cur_number_chars)))
          for pni in cur_number_char_indices:
            found_part_number_indices.append(pni)
  return found_part_numbers


In [4]:
def get_part_ratios(formatted_inp):

  def is_gear(x):
    return x == '*'

  max_i =  max([x[0] for x in formatted_inp.keys()])
  max_j = max([x[1] for x in formatted_inp.keys()])

  def check_adjacent_symbols(i, j):
    for ii in [i-1,i,i+1]:
      for jj in [j-1, j, j+1]:
        if not (ii == i and jj == j):
          if is_gear(formatted_inp.get((ii,jj), None)):
            return ((ii,jj))
    return False

  gear_to_numbers = {}
  found_part_numbers = []
  found_part_number_indices = []
  for i in range(max_i+1):
    for j in range(max_j+1):
      if is_number(formatted_inp.get((i,j), None)):
        k = j
        cur_number_chars = []
        cur_number_char_indices = []
        adjacent_symbol = False
        adjacent_gear_loc = None
        while (is_number(formatted_inp.get((i,k), None))
        and (i,k) not in found_part_number_indices):
          cur_number_chars.append(formatted_inp[(i,k)])
          cur_number_char_indices.append((i,k))
          if(check_adjacent_symbols(i, k) is not False):
            adjacent_symbol = True
            adjacent_gear_loc = check_adjacent_symbols(i, k)
          k += 1
        if adjacent_symbol:
          found_part_numbers.append(int(''.join(cur_number_chars)))
          if adjacent_gear_loc not in gear_to_numbers.keys():
            gear_to_numbers[adjacent_gear_loc] = []
          gear_to_numbers[adjacent_gear_loc].append(int(''.join(cur_number_chars)))
          for pni in cur_number_char_indices:
            found_part_number_indices.append(pni)
  # Filter out only gears with 2 adjacent numbers
  gear_to_numbers = dict(filter(lambda p: len(p[1])==2, gear_to_numbers.items()))

  gear_ratios = [x[0] * x[1] for x  in gear_to_numbers.values()]
  return gear_ratios

In [5]:
import time

t = time.time()

formatted_inp = format_input(inp)

print(sum(get_part_numbers(formatted_inp)))
print(sum(get_part_ratios(formatted_inp)))

print("\nRUNTIME: ", time.time()-t)

546563
91031374

RUNTIME:  0.35807132720947266
